# Lecture 2: NumPy routines
In this lecture we will discuss various useful routines and mathematical functions within NumPy.

## Contents
- [Functions](#function)
    - [Pure functions](#Pure-functions)
    - [Functions for arrays](#arrays)
    - [Missing arguments](#missing)
- [Arrays in conditions](#condition)
- [Random numbers](#Random-numbers)
    - [Random seed](#seed)
    - [Random subsets](#randomsubsets)
- [Mathematical functions](#Mathematical-functions)
    - [Special numbers](#special)
    - [Matrix algebra](#matrixalgebra)
    - [Advanced functions for arrays](#advanced)   
- [Linear algebra](#Linear-algebra)
    - [Equation solving](#equationsolving)
    - [Least-squares method](#leastsquares)
    - [Matrix decompositions](#matrixdecompositions)
- [Sorting and searching](#Sorting-and-searching)
- [Enumeration of arrays](#enumeration)

In [2]:
import numpy as np
np.set_printoptions(precision=2,suppress=True) 

You can use the `np.set_printoptions()` comment above to specify the desired number of decimals that you want see in numerical output. The option `suppress=True` suppresses scientific notation (the 'e-10' or 'e10' part you sometimes see).

## Functions <a name="function"></a>
Before we dive a bit more into NumPy functions, we discuss some general concepts regarding functions.

### Pure functions

Pure functions are functions that do not produce any side effects. This concept is not just relevant for Numpy arrays, but for all mutable objects (i.e., objects that can be modified) in Python such as lists and dictionaries.

A function does not have any side effect if it does not change the state of the program. Given the same input, a pure function will always produce the same output. In general, it is better to write pure functions instead of functions with side effect, because the latter are often confusing to users.

Consider the following two functions that returns an array with all of the negative entries doubled.

In [3]:
def double_negative_side_effect(x):
    """Return array with all negative entries doubled."""
    y = x  # y is simply a view of x
    mask = y < 0
    y[mask] = y[mask] * 2  # also changes its input argument `x`
    return y

def double_negative_pure(x):
    """Return array with all negative entries doubled."""
    y = x.copy()  # make independent copy of x
    mask = y < 0
    y[mask] = y[mask] * 2
    return y

In [7]:
# Initialize some matrix.
X = np.array([[-1,-2,5,4,3],[-4,-6,8,-10,1],[-6,-3,-6,8,1]])
print('X = \n',X)

X = 
 [[ -1  -2   5   4   3]
 [ -4  -6   8 -10   1]
 [ -6  -3  -6   8   1]]


In [10]:
print('X = \n',X)
Y = double_negative_side_effect(X)
print('Y = \n', Y)
print('X = \n',X)  # x has changed as well

Z = double_negative_side_effect(X)
print('Z = \n',Z)
print('X = \n',X)  # ...and keeps changing

X = 
 [[ -1  -2   5   4   3]
 [ -4  -6   8 -10   1]
 [ -6  -3  -6   8   1]]
Y = 
 [[ -2  -4   5   4   3]
 [ -8 -12   8 -20   1]
 [-12  -6 -12   8   1]]
X = 
 [[ -2  -4   5   4   3]
 [ -8 -12   8 -20   1]
 [-12  -6 -12   8   1]]
Z = 
 [[ -4  -8   5   4   3]
 [-16 -24   8 -40   1]
 [-24 -12 -24   8   1]]
X = 
 [[ -4  -8   5   4   3]
 [-16 -24   8 -40   1]
 [-24 -12 -24   8   1]]


In [11]:
# The pure function does not change `x` and always returns
# the same result.
X = np.array([[-1,-2,5,4,3],[-4,-6,8,-10,1],[-6,-3,-6,8,1]])
print('X = \n',X)

Y = double_negative_pure(X)
print('Y = \n',Y)
print('X = \n',X) #x did not change

X = 
 [[ -1  -2   5   4   3]
 [ -4  -6   8 -10   1]
 [ -6  -3  -6   8   1]]
Y = 
 [[ -2  -4   5   4   3]
 [ -8 -12   8 -20   1]
 [-12  -6 -12   8   1]]
X = 
 [[ -1  -2   5   4   3]
 [ -4  -6   8 -10   1]
 [ -6  -3  -6   8   1]]


As mentioned several times by now, to get good performance we should try to avoid looping over elements in our vectors and matrices (i.e., arrays), and instead use vectorized algorithms. The first step in converting a scalar algorithm to a vectorized algorithm is to make sure that the functions we write work with vector inputs. Consider the following function.

### Functions for arrays <a name="arrays"></a>

In [14]:
def theta_1(x):
    """
    Scalar implementation of the Heaviside step function.
    """
    if x >= 0:
        return 1
    else:
        return 0

In [15]:
try:
    print(theta_1(np.array([-3,-2,-1,0,1,2,3])))
except Exception as err:
    print(err)

The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


OK, that didn't work because we didn't write the `theta_1` function so that it can handle a vector input... We have to rewrite our function to work with `numpy` arrays.

In [16]:
def theta_2(x):
    """
    Array implementation of the Heaviside step function.
    """
    # There are several alternatives to accomplish this in numpy.
    # return np.where(x >= 0, 1, 0)
    # return (x >= 0).astype(int)
    return (x >= 0) * 1

In [17]:
x = theta_2(np.array([-3,-2,-1,0,1,2,3]))
print(x)

[0 0 0 1 1 1 1]


In [19]:
# still works for scalars as well
print(theta_2(-1.2))
print(theta_2(2.6))

0
1


### Missing arguments <a name="missing"></a>
It is possible to write functions in such a way that missing arguments are set to a given default if they are not inputted. Consider the function 

$$
f(x,a,b,c) = a\cdot x^2 + b \cdot x + c.
$$

The goal is to write a function that evaluates this function in a given point $(x,a,b,c)$. 

In [20]:
def f(x,a,b,c):
    return a*x^2 + b*x + c

In [21]:
print(f(1,1,2,1))

4


Now suppose that we define $g(x,c) = f(x,a,1,2)$ as the function that always has $a = 1$ and $b = 2$. A simple way to do this (while remaining somewhat flexible if we want to change the fixed values of $a$ and $b$) is to give a default  value to $a$ and $b$ in the function $f$ above.

In order to accomplish this, we have to change the order of the variables, since a function must always have non-default arguments before arguments with a default. If we would not change the order of the variables, but write `g(x,a=1,b=2,c)` we get a `SyntaxError`.

In [22]:
def g(x,c,a=1,b=2):
    return a*x^2 + b*x + c

In [24]:
print(g(1,1))

4


Furthermore, it is also possible to assign a default value based on the arguments that were inputted. For example, we could set $a = 2c$ and $b = c + 1$.

In [25]:
def h(x,c,a=None,b=None):
    if a == None:
        a = 2*c
    if b == None:
        b = c + 1
    return a*x^2 + b*x + c

In [26]:
print(h(1,1))

7


In [27]:
print(h(1,1,1,2)) #Equals f(1,1,2,1) since argument order has been changed.

4


## Arrays in conditions <a name="conditions"></a>

When using arrays in conditions,for example `if` statements and other boolean expressions, one needs to use `any` or `all`, which requires that any or all elements in the array evalutes to `True`:

In [29]:
M = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])

In [30]:
F = M > 5
print(F)

[[False False False False]
 [False  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [31]:
F = (M > 5).any()
print(F)
G = (M > 5).all()
print(G)

True
False


In [33]:
if (M > 5).any():
    print('At least one element in M is larger than 5')
else:
    print('No element in M is larger than 5')

At least one element in M is larger than 5


In [34]:
if (M > 5).all():
    print('All elements in M are larger than 5')
else:
    print('Not all elements in M are larger than 5')

Not all elements in M are larger than 5


In [35]:
F = (M > 5).all(axis=0)  # check along axis=0
print(F)

[False False False False]


In [36]:
F = (M > 5).all(axis=1)  # check along axis=1
print(F)

[False False  True  True]


## Random numbers

In order to generated random numbers, one can use the `numpy.random` module. It can be used to generated numbers uniformly from a (discrete) interval, as well as from the normal distribution. The module can also be used for many other types of distributions. The `scipy` module can also be used for this, which might be a better choice if you also need other properties of probability distributions. The most common functions are
- `np.random.rand`: uniform numbers over $[0, 1)$
- `np.random.randn`: standard normal random numbers (i.e., with mean 0 and standard deviation 1)
- `np.random.randint`: uniform random integers from interval $\{a,a+1,...,b\}$ for integers $a$ and $b$

In [37]:
x = np.random.rand()
print(x)

0.6187570955540956


In [38]:
X = np.random.rand(2, 4) #the input parameters determine the size of the matrix
print(X)

[[0.08 0.56 0.24 0.94]
 [0.4  0.43 0.19 0.94]]


In [40]:
x = np.random.randn(6) #one input parameter gives one-dimensional array
print(x)

[-0.58  0.85 -0.22 -1.05  0.86  1.03]


In [41]:
X = np.random.randint(10, 20, (4, 3))  #matrix of size 4 x 3 with random integers from {10, 11, ..., 19} 
print(X)

[[12 11 13]
 [13 16 16]
 [17 16 18]
 [10 12 19]]


If it fits in memory, then it's better to generate random numbers in large blocks.

In [42]:
def slow_rand(n):
    """Return array of uniform random numbers.
    
    Call np.random.rand for each individual entry.
    """
    x = np.empty(n)
    for i in range(n):
        x[i] = np.random.rand()
    return x

In [43]:
%timeit slow_rand(10000) #timeit(a) can be used to run a piece of code a times. It displays average running time.

3.73 ms ± 36.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%timeit np.random.rand(10_000)

57.7 µs ± 476 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Random seed <a name="seed"></a>
It is possible to ''fix'' random numbers generated by Python, so that if you would run the same 'random' command again, you obtain exactly the same output. For this you can use `np.random.seed`. 

Let us look at an example. Rerunning the code below will give (almost surely) a different outcome every time. Try this.

In [45]:
#We generate a random integer from the range {0,...,999}.
print(np.random.randint(1000))

215


We next first fix the seed of the (pseudo)-random number generator (you can choose any number other than 3 here). The output of the command below will always be the same, no matter how often you rerun this piece of code.


In [46]:
np.random.seed(3) 
print(np.random.randint(1000))

874


You can also use multiple random seeds if you want. This can be done with `np.random.RandomState`.

In [47]:
#Rerunning the code below always gives the same two values of exponential(10) (based on the respective random seed).
y1 = np.random.RandomState(10)
y2 = np.random.RandomState(20)

x1 = y1.exponential(10)
x2 = y2.exponential(10)
print(x1,x2)

14.754344455461826 8.870494583824275


### Random subsets <a name="randomsubsets"></a>
It is also possible to generate (uniform) random subsets of a given size $m$ of some ground set $\{0,1,\dots,n-1\}$. Uniform here means that every subset $K \subseteq \{0,1,\dots,n-1\}$ of size $m$ is equally likely to be chosen. You can use either the `sample()` or `choices()` functions from the `random` module, depending on whether you want to choose a subset $K$ with or without replacement (i.e., whether or not elements may appear multiple times in the set $K$, or whether all elements have to be distinct).

In [48]:
import random

In [49]:
n = 10
m = 4
K = random.sample(range(n),k=m)
print(K)

[6, 3, 5, 1]


In [50]:
#Chooses 11 elements from list of 10 elements so at least one elements appears twice
m = 11
K = random.choices(np.arange(0,10),k=m) 
print(K)

[5, 7, 9, 4, 4, 6, 4, 3, 9, 1, 7]


## Mathematical functions

Most of the mathematical functions provided by the standard Python library `math` and many more are available within `numpy` to be applied to arrays. Just search in the documentation for the function you need. 
Some function have optional arguments that can change the behavior of the function, such working as along an axis.
Here we list just a few:
- `sin`, `cos`, `tan`
- `exp`, `log`, `log10`, `log2`
- `sum`, `cumsum`, `prod`, `cumprod`
- `around`, `floor`, `ceil`
- `mod`, `divmod`
- `sqrt`, `abs`, `power`

### Special numbers <a name="special"></a>

In Numpy, we can use `np.inf` and `np.nan` to represent *infinity* and *not a number*, which can sometimes be the result of some mathematical operation.

In [52]:
print(np.nan)

nan


In [53]:
print(np.inf)
print(-np.inf)

inf
-inf


In [54]:
x = np.array([-1, 0, 1])

In [55]:
print(x / 0)

[-inf  nan  inf]


<ipython-input-55-bd6c4c1405a7>:1: RuntimeWarning: divide by zero encountered in true_divide
  print(x / 0)
<ipython-input-55-bd6c4c1405a7>:1: RuntimeWarning: invalid value encountered in true_divide
  print(x / 0)


In [56]:
print(np.log(0))
print(np.log(-1))

-inf
nan


<ipython-input-56-ef2a659a346d>:1: RuntimeWarning: divide by zero encountered in log
  print(np.log(0))
<ipython-input-56-ef2a659a346d>:2: RuntimeWarning: invalid value encountered in log
  print(np.log(-1))


### Matrix algebra <a name="matrixalgebra"></a>

What about matrix mutiplication? There are two ways. We can either use the `dot` function, which applies a matrix-matrix, matrix-vector, or inner vector multiplication to its two arguments: 

In [58]:
v1 = np.arange(0, 4)
M = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])

In [59]:
print(np.dot(M,M))

[[ 90 100 110 120]
 [202 228 254 280]
 [314 356 398 440]
 [426 484 542 600]]


In [60]:
print(np.dot(M, v1))

[20 44 68 92]


In [61]:
print(np.dot(v1, v1))

14


Alternatively, we can use the `@` operator:

In [62]:
print(M @ M)

[[ 90 100 110 120]
 [202 228 254 280]
 [314 356 398 440]
 [426 484 542 600]]


In [63]:
print(M @ v1)

[20 44 68 92]


In [64]:
print(v1 @ v1)

14


See also the related functions: `inner`, `outer`, `cross`, `kron`, `tensordot`.

Note that in `numpy`, you can also create `matrix` objects, which can be used to represent a matrix (obviously..). However, you are adviced to simply use a 2-dimensional `ndarray` to represent a matrix in `numpy` and not to use this `matrix` object (the class `numpy.matrix` might be removed in the future).

### Advanced functions for arrays <a name="advanced"></a>
Below we give some more advanced operations related to addition and multiplication that can be performed on arrays.

#### sum, prod, and trace

In [72]:
d = np.arange(1,10)
print(d)

[1 2 3 4 5 6 7 8 9]


In [73]:
# sum up all elements
print(np.sum(d))
print(d.sum())

45
45


In [74]:
# product of all elements
print(np.prod(d+1))

3628800


In [77]:
# cumulative sum. If d = [d1 d2 d3] you get [d1 (d1+d2) (d1+d2+d3)]
print(np.cumsum(d))

[ 1  3  6 10 15 21 28 36 45]


In [78]:
# cumulative product. Same as np.cumsum, but not with product instead of sum
print(np.cumprod(d))

[     1      2      6     24    120    720   5040  40320 362880]


In [79]:
# same as: diag(A).sum()
print(np.trace(M))

34


## Linear algebra

The module `np.linalg` contains several standard linear algebra algorithms.

For example,
- `det`: matrix determinant
- `inv`: matrix inverse
- `norm`: matrix or vector norm
- `matrix_rank`: matrix rank
- `trace`: trace of matrix

### Equation solving <a name="equationsolving"></a>

You can use `np.linalg.solve` to solve a square linear system $Ax = b$ where $A \in \mathbb{R}^{n \times n}$ and $b \in \mathbb{R}^n$. If A is singular, meaning its columns are linearly dependent, and, hence, its inverse $A^{-1}$ does not exist, `np.linalg` will raise a `LinAlgError`.

In [80]:
A = np.array([[1,1],[1,1]]) #n x n  matrix which is singular
b = np.array([1,1])

try:
    x = np.linalg.solve(A, b)
    print(x)
except np.linalg.LinAlgError:
    print('The system Ax = b has no solution, as A is a singular matrix')

The system Ax = b has no solution, as A is a singular matrix


If we randomly generate the entries of $A$ and $b$, the matrix $A$ is usually not singular. (This is based on the mathematical fact that a matrix with random entries is 'with high probability' not singular.)

In [87]:
# Solve a square linear system of equations.
n = 4
A = np.random.randn(n,n)
b = np.random.randn(n)
print('A = \n', A)
print('b = \n', b)

A = 
 [[ 0.48 -0.3  -0.42 -0.86]
 [-0.41  1.25 -1.83 -1.6 ]
 [ 0.69  0.93  1.59  0.4 ]
 [-1.27  0.47  1.3   1.23]]
b = 
 [ 0.23  0.27 -0.67 -0.2 ]


In [88]:
try:
    x = np.linalg.solve(A, b)
    print(x)
except np.linalg.LinAlgError:
    print('The system $Ax = b$ has no solution, as $A$ is a singular matrix')

[-0.29 -0.31 -0.03 -0.3 ]


In [89]:
# This should be equal to `b`.
print(A @ x)

[ 0.23  0.27 -0.67 -0.2 ]


You can use `np.allclose` to determine if two vectors are (up to small numerical error) equal.

In [90]:
# Verify that we really have a solution (accepting small numerical errors).
print(np.allclose(A @ x, b))

True


The numerical error allowed for checking whether two arrays are the same can also be specified in `np.allclose`. You can set both the relative tolerance (rtol) and the absolute tolerance (atol). See the documentation of `np.allclose` for details on what these notions mean mathematically.

In [91]:
print(np.allclose(A @ x, b, rtol=1e-10, atol=1e-20))

True


In [92]:
print(np.allclose(A @ x, b, rtol=1e-20, atol=1e-20)) #We require higher precision than before.

False


### Least-squares method <a name="leastsquares"></a>
If we have an $A \in \mathbb{R}^{m \times n}$ and vector $b \in \mathbb{R}^n$ for which the linear system $Ax = b$ does not have a solution, we can use the *least-squares* function `np.linalg.lstsq` to find a vector $x$ that minimizes $||Ax - b||_2$. Note that this also works if $A$ is not a square matrix (i.e., $m \neq n$).

The function `np.linalg.lstsq` returns four objects of which the first two are: 

 - [0]: The least-square solution $\hat{x}$. 
 - [1]: The value of $||A\hat{x} - b||^2_2$.

See the <a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html">documentation</a> of `np.linalg.lstsq` for all details.

In [104]:
m, n = 10, 4
A = np.random.randn(m, n)
x = np.random.randn(n)  
b = A @ x #Under these definitions x satisfies Ax = b (can check this with np.allclose(A@x,b))

Suppose now we have some 'noise' in the right hand side vector b, resulting in the vector $b_{noise}$. Then $Ax \neq b_{noise}$, i.e., $x$ is no longer a solution of the linear system.

In [105]:
b_noise = b + 0.1 * np.random.randn(m) 

In [106]:
# Solve least-squares problem.
import numpy as np

result = np.linalg.lstsq(A, b_noise, rcond=None)
print(result) 

# The least-squares solution is given by first output object of np.linalg.lstsq
x_noise = result[0]

print('x vs. x_noise:')
print(np.stack((x, x_noise), axis=1)) #np.stack is (yet) another way of stacking (see Lecture 1) two vectors

(array([-0.12, -0.44, -0.41,  0.87]), array([0.02]), 4, array([5.66, 5.33, 3.49, 2.33]))
x vs. x_noise:
[[-0.09 -0.12]
 [-0.47 -0.44]
 [-0.42 -0.41]
 [ 0.87  0.87]]


As we are only interested in the first object return by  `np.linalg.lstsq`, we can alternatively also suppress the remaining outputs. Suppressing certain outputs can be done for all functions that return multiple outputs.

In [107]:
x2_noise, *_ = np.linalg.lstsq(A, b_noise, rcond=None) #We suppress all outputs, except the first one.

print('x vs. x_noise:')
print(np.stack((x, x_noise), axis=1))

x vs. x_noise:
[[-0.09 -0.12]
 [-0.47 -0.44]
 [-0.42 -0.41]
 [ 0.87  0.87]]


### Matrix decompositions <a name="matrixdecompositions"></a>

- `eig`: eigenvalues and eigenvectors square matrix
- `eigh`: eigenvalues and eigenvectors *symmetric* matrix
- `qr`: QR decomposition
- `svd`: singular value decomposition

In [109]:
# Create symmetric matrix
b = np.random.randn(4, 4)
A = b @ b.T
print(A) #Note that A is symmetric

[[ 3.09  0.47 -0.55  3.22]
 [ 0.47  0.85 -0.08  0.55]
 [-0.55 -0.08  1.   -1.76]
 [ 3.22  0.55 -1.76  5.1 ]]


In [111]:
eig_vals, eig_vectors = np.linalg.eigh(A)

#Only obtain eigenvalues (first output argument)
eigenvalues, *_ = np.linalg.eigh(A)

#Only obtain eigenvectors (second output argument)
*_, eigenvectors = np.linalg.eigh(A)

In [112]:
# Eigenvalues
print(eig_vals)

array([0.05, 0.75, 1.25, 7.99])

In [113]:
# Eigenvectors
print(eig_vectors)

[[-0.42  0.24 -0.68 -0.56]
 [-0.05 -0.97 -0.23 -0.1 ]
 [ 0.73  0.09 -0.63  0.24]
 [ 0.53 -0.02  0.31 -0.79]]


In [114]:
# Verify A * v = lambda * v
b = np.allclose(A @ eig_vectors[:, 0], eig_vals[0] * eig_vectors[:, 0])
print(b)

True


## Sorting and searching
Some of the functions below we also saw in the first lecture already.

In [118]:
X = np.random.rand(8, 4)
print(X)

[[0.96 0.81 0.25 0.91]
 [0.52 0.52 0.73 0.88]
 [0.64 0.1  0.72 0.46]
 [0.4  0.32 0.47 0.87]
 [0.94 0.28 0.84 0.49]
 [0.27 0.25 0.64 0.41]
 [0.83 0.44 0.84 0.54]
 [0.89 0.78 0.28 0.15]]


In [119]:
# sort columns, i.e., sort along axis=0
X_sorted = np.sort(X, axis=0)
print(X_sorted)

[[0.27 0.1  0.25 0.15]
 [0.4  0.25 0.28 0.41]
 [0.52 0.28 0.47 0.46]
 [0.64 0.32 0.64 0.49]
 [0.83 0.44 0.72 0.54]
 [0.89 0.52 0.73 0.87]
 [0.94 0.78 0.84 0.88]
 [0.96 0.81 0.84 0.91]]


In [125]:
# return indices that would sort array along axis=1
Y = np.argsort(X, axis=1)
print(Y)

[[2 1 3 0]
 [0 1 2 3]
 [1 3 0 2]
 [1 0 2 3]
 [1 3 2 0]
 [1 0 3 2]
 [1 3 0 2]
 [3 2 1 0]]


In [126]:
# Maximum of column (along axis=0)
z = np.max(x, axis=0)
print(z)

[0.86 0.99 0.87 0.94]


In [127]:
# Indices of maximum locations per column
idx = np.argmax(x, axis=0)
print(idx)

[5 6 1 5]


In [129]:
# Use index to get maximum values
max_col = x[idx, [0, 1, 2, 3]]
print(max_col) #Same as np.max(x, axis=0)

[0.86 0.99 0.87 0.94]


## Enumeration of arrays <a name="enumeration"></a>

Another useful package of NumPy is the `itertools` module that can be used to iterate (dynamically) over objects. 

In [133]:
import itertools as it

Suppose we want to print all arrays of length $3$ whose entries are integers from the set $\{0,1,2\}$. One way to do this is by using three `for`-loops.

In [134]:
for i in range(3):
    for j in range(3):
        for k in range(3):
            x = np.array([i,j,k])
            print(x)

[0 0 0]
[0 0 1]
[0 0 2]
[0 1 0]
[0 1 1]
[0 1 2]
[0 2 0]
[0 2 1]
[0 2 2]
[1 0 0]
[1 0 1]
[1 0 2]
[1 1 0]
[1 1 1]
[1 1 2]
[1 2 0]
[1 2 1]
[1 2 2]
[2 0 0]
[2 0 1]
[2 0 2]
[2 1 0]
[2 1 1]
[2 1 2]
[2 2 0]
[2 2 1]
[2 2 2]


Now suppose we want to print all such arrays of length $4$ (or more general length $k$) whose elements are integers from the set $\{0,1,2\}$. In order to do this, we would have to keep adding `for`-loops to the code above. Alternatively, we could write a <i>recursive</i> function (that calls on itself), but this is beyond the scope of this course. 

The `itertools` module can be used to perform this enumeration task by calling on the function `product`.

In [135]:
n = 4 #elements are integers from set {0,1,..,n-1}
k = 3 #length of the vector

for x in it.product(range(n),repeat=k):
    y = np.array(x) #it.product returns a tuple
    print(y)

[0 0 0]
[0 0 1]
[0 0 2]
[0 0 3]
[0 1 0]
[0 1 1]
[0 1 2]
[0 1 3]
[0 2 0]
[0 2 1]
[0 2 2]
[0 2 3]
[0 3 0]
[0 3 1]
[0 3 2]
[0 3 3]
[1 0 0]
[1 0 1]
[1 0 2]
[1 0 3]
[1 1 0]
[1 1 1]
[1 1 2]
[1 1 3]
[1 2 0]
[1 2 1]
[1 2 2]
[1 2 3]
[1 3 0]
[1 3 1]
[1 3 2]
[1 3 3]
[2 0 0]
[2 0 1]
[2 0 2]
[2 0 3]
[2 1 0]
[2 1 1]
[2 1 2]
[2 1 3]
[2 2 0]
[2 2 1]
[2 2 2]
[2 2 3]
[2 3 0]
[2 3 1]
[2 3 2]
[2 3 3]
[3 0 0]
[3 0 1]
[3 0 2]
[3 0 3]
[3 1 0]
[3 1 1]
[3 1 2]
[3 1 3]
[3 2 0]
[3 2 1]
[3 2 2]
[3 2 3]
[3 3 0]
[3 3 1]
[3 3 2]
[3 3 3]


In [136]:
n = 4 #elements are integers from set {0,1,..,n-1}
k = 2 #length of the vector

for x in it.product(range(n+1),repeat=k):
    y = np.array(x) #it.product returns a tuple
    print(y)

[0 0]
[0 1]
[0 2]
[0 3]
[0 4]
[1 0]
[1 1]
[1 2]
[1 3]
[1 4]
[2 0]
[2 1]
[2 2]
[2 3]
[2 4]
[3 0]
[3 1]
[3 2]
[3 3]
[3 4]
[4 0]
[4 1]
[4 2]
[4 3]
[4 4]


In [137]:
n = 4
k = 3

for x in it.product(range(n+1),repeat=k):
    y = np.array(x)#it.product returns a tuple
    if np.sum(y) == n: 
        print(y)

[0 0 4]
[0 1 3]
[0 2 2]
[0 3 1]
[0 4 0]
[1 0 3]
[1 1 2]
[1 2 1]
[1 3 0]
[2 0 2]
[2 1 1]
[2 2 0]
[3 0 1]
[3 1 0]
[4 0 0]


It is also possible to specify explicit ranges for the integer values of the elements on a given position. The code below returns all arrays whose first element lies in the range $\{0,1,2\}$, whose second element lies in the range $\{2,3\}$ and whose third elements lies in the range $\{1,2,3,4\}$.

In [138]:
for x in it.product(range(3),range(2,4),range(1,5)):
    y = np.array(x) #it.product returns a tuple
    print(y)

[0 2 1]
[0 2 2]
[0 2 3]
[0 2 4]
[0 3 1]
[0 3 2]
[0 3 3]
[0 3 4]
[1 2 1]
[1 2 2]
[1 2 3]
[1 2 4]
[1 3 1]
[1 3 2]
[1 3 3]
[1 3 4]
[2 2 1]
[2 2 2]
[2 2 3]
[2 2 4]
[2 3 1]
[2 3 2]
[2 3 3]
[2 3 4]
